In [16]:
# To continue work on this, maybe make it so that in the event of certain emotions, certain tabs open which can be accessed

from flask import Flask, render_template_string, Response, request
import cv2
from deepface import DeepFace
import time
import threading
import logging

app = Flask(__name__)

logging.basicConfig(filename='app.log', level=logging.INFO)

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

@app.route('/ping', methods=['GET'])
def ping():
    return "pong"

@app.route('/')
def index():
    template_str = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <title>Emotion Checker</title>
        <style>
            body {
                font-family: Arial, sans-serif;  
                display: flex;
                justify-content: center; /* Center horizontally */
                align-items: center; /* Center vertically */
                height: 100vh; /* 100% of the viewport height */
                margin: 0; /* Remove default margin */
            }

            #emotionText {
                text-align: center; 
                padding: 20px; 
            }
        </style>
    </head>
    <body>
        <div id="emotionText" style="font-size: 120px;"></div>
        <script>
            function updateBackgroundColor() {
                fetch('/get_emotion').then(response => response.text()).then(emotion => {
                    let color;
                    switch (emotion) {
                        case 'happy':
                            color = 'yellow';
                            break;
                        case 'sad':
                            color = 'blue';
                            break;
                        case 'angry':
                            color = 'red';
                            break;
                        case 'neutral':
                            color = 'gray';
                            break;
                        case 'surprise':
                            color = 'pink';
                            break;
                        case 'fear':
                            color = 'purple';
                            break;
                        case 'disgust':
                            color = 'green';
                            break;
                        default:
                            color = 'white'; // default or 'error' color
                    }
                    document.body.style.backgroundColor = color;
                    
                    // Update the text content
                    let emotionTextElement = document.getElementById('emotionText');
                    emotionTextElement.textContent = `${emotion}`;
                });
            }

            setInterval(updateBackgroundColor, 300);
        </script>
    </body>
    </html>
    """
    return render_template_string(template_str)

# @app.route('/video_feed')
# def video_feed():
#     def generate():
#         while True:
#             ret, frame = cap.read()
#             try:
#                 result = DeepFace.analyze(frame, actions=['emotion'])
#             except ValueError as e:
#                 print(e)
#                 continue

#             gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#             faces = faceCascade.detectMultiScale(gray, 1.1, 4)
#             for (x, y, w, h) in faces:
#                 cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

#             font = cv2.FONT_HERSHEY_SIMPLEX
#             cv2.putText(frame, result[0]['dominant_emotion'], (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)

#             ret, jpeg = cv2.imencode('.jpg', frame)
#             yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')
            
#     return Response(generate(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/get_emotion', methods=['GET'])
def get_emotion():
    global last_request_time
    last_request_time = time.time()  # Update the timestamp

    ret, frame = cap.read()
    try:
        result = DeepFace.analyze(frame, actions=['emotion'])
        dominant_emotion = result[0]['dominant_emotion']
        logging.info(f"Emotion detected: {dominant_emotion}")
    except ValueError:
        dominant_emotion = 'error'
        logging.error("Error in emotion detection.")
    return dominant_emotion

last_request_time = time.time()

def shutdown_server():
    """Function to shut down the Flask server."""
    print("Shutting down...")
    cap.release()
    func = request.environ.get('werkzeug.server.shutdown')
    if func:
        func()
        
@app.route('/shutdown', methods=['POST'])
def shutdown():
    shutdown_server()
    return 'Server shutting down...'

@app.errorhandler(404)
def page_not_found(e):
    return "404 - Page Not Found", 404

@app.errorhandler(500)
def server_error(e):
    return "500 - Internal Server Error", 500

@app.route('/some_route')  # Adjusted this route
def some_route():
    logging.info("Some route accessed")
    return "Some Route"

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://169.231.37.144:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Oct/2023 14:38:17] "GET / HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 27.48it/s]
127.0.0.1 - - [08/Oct/2023 14:38:19] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 19.55it/s]
127.0.0.1 - - [08/Oct/2023 14:38:19] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 20.79it/s]
127.0.0.1 - - [08/Oct/2023 14:38:19] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 16.93it/s]
127.0.0.1 - - [08/Oct/2023 14:38:20] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 21.32it/s]
127.0.0.1 - - [08/Oct/2023 14:38:20] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 18.92it/s]
127.0.0.1 - - [08/Oct/2023 14:38:21] "GET /get_emotion HTTP/1.1" 200 -


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 21.46it/s]
127.0.0.1 - - [08/Oct/2023 14:38:47] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 15.28it/s]
127.0.0.1 - - [08/Oct/2023 14:38:48] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 15.89it/s]
127.0.0.1 - - [08/Oct/2023 14:38:48] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 13.94it/s]
127.0.0.1 - - [08/Oct/2023 14:38:49] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 19.11it/s]
127.0.0.1 - - [08/Oct/2023 14:38:49] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 21.41it/s]
127.0.0.1 - - [08/Oct/2023 14:38:50] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 20.84it/s]
127.0.0.1 - - [08/Oct/2023 14:38:50] "GET /get_emotion HTTP/1.1" 200 -
Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 15.83it/s]

Shutting down...


Exception in thread Thread-1108 (monitor_client_activity):
Traceback (most recent call last):
  File "C:\Users\abbra\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\abbra\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\abbra\AppData\Local\Temp\ipykernel_17440\1191511927.py", line 146, in monitor_client_activity
  File "C:\Users\abbra\AppData\Local\Temp\ipykernel_17440\1191511927.py", line 131, in shutdown_server
  File "C:\Users\abbra\anaconda3\Lib\site-packages\werkzeug\local.py", line 316, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abbra\anaconda3\Lib\site-packages\werkzeug\local.py", line 513, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Con

In [14]:

# from flask import Flask, render_template_string
# import cv2
# from deepface import DeepFace
# import threading

# app = Flask(__name__)

# faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# cap = cv2.VideoCapture(1)

# @app.route('/')
# def index():
#     # Your HTML template string (simplified here)
#     template_str = """
#     <!DOCTYPE html>
#     <html lang="en">
#     <head>
#         <title>OpenCV with Flask</title>
#     </head>
#     <body>
#         <script>
#             // Your JavaScript to update the background color
#         </script>
#     </body>
#     </html>
#     """
#     return render_template_string(template_str)

# @app.route('/get_emotion', methods=['GET'])
# def get_emotion():
#     ret, frame = cap.read()

#     try:
#         result = DeepFace.analyze(frame, actions=['emotion'])
#         dominant_emotion = result['dominant_emotion']
#     except ValueError:
#         dominant_emotion = 'error'
    
#     return dominant_emotion

# def run():
#     app.run(use_reloader=False, debug=False)

# # Running Flask in a thread
# threading.Thread(target=run).start()